# © Copyright 2018 Aaron Ma. All Rights Reserved. #

If you’ve ever had your blood drawn, chances are that you’ve had a blood test where your doctor looked up the following statistics about your blood among others:

* The total number of White Blood Cells (WBCs) in your blood stream.
* The number of Neutrophils, Lymphocytes, Basophils, and Eosinophils (all types of WBCs) in your cell. This is known as a differentiated blood cell count.

The density of WBCs in our blood stream provides a glimpse into the state of our immune system and any potential risks we might be facing. In particular, a dramatic change in the WBC count relative to your baseline is generally a sign that your body is currently being affected by an antigen. Moreover, a variation in a specific type of WBC generally correlates with a specific type of antigen.

![Retry?](https://cdn-images-1.medium.com/max/1600/1*uUW8GaUu3gCJN0XIndMDTg.png)

## Dataset
![Retry?](https://cdn-images-1.medium.com/max/2000/1*5-X837TmpQj3QLyDX3ow3Q.png)

```python
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt
```

```python
epochs = 20
BASE_DIR = './train_tmp'
batch_size = 32
```

```python
def get_model():
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])

    return model

model = get_model()
print(model.summary())

def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []

    for wbc_type in os.listdir(folder):
        if not wbc_type.startswith('.'):
            if wbc_type in ['NEUTROPHIL', 'EOSINOPHIL']:
                label = 'POLYNUCLEAR'
            else:
                label = 'MONONUCLEAR'
            for image_filename in os.listdir(folder + wbc_type):
                img_file = cv2.imread(folder + wbc_type + '/' + image_filename)
                if img_file is not None:
                    # Downsample the image to 120, 160, 3
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y
```

![Retry?](https://raw.githubusercontent.com/aaronhma/google-images/master/do.png)

```python
X_train, y_train = get_data(BASE_DIR + 'images/TRAIN/')
X_test, y_test = get_data(BASE_DIR + 'images/TEST_SIMPLE/')

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

model = get_model()

# fits the model on batches
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=epochs,
    shuffle=True,
    batch_size=batch_size)

model.save_weights('binary_model.h5')
```
```bash
Train on 7965 samples, validate on 1992 samples
Epoch 1/20
7965/7965 [==============================] - 13s - loss: 0.6959 - acc: 0.6244 - val_loss: 0.9353 - val_acc: 0.0000e+00 - ETA: 13s - loss: 0.8030 - acc: 0.6090 - ETA: 4s - loss: 0.7107 - acc: 0.6206
Epoch 2/20
7965/7965 [==============================] - 13s - loss: 0.6529 - acc: 0.6288 - val_loss: 1.1067 - val_acc: 0.0000e+00
Epoch 3/20
7965/7965 [==============================] - 13s - loss: 0.5241 - acc: 0.7234 - val_loss: 0.4966 - val_acc: 0.8725
Epoch 4/20
7965/7965 [==============================] - 13s - loss: 0.3728 - acc: 0.8562 - val_loss: 0.7631 - val_acc: 0.5718 - ETA: 12s - loss: 0.4190 - acc: 0.8125 - ETA: 6s - loss: 0.4005 - acc: 0.8435
Epoch 5/20
7965/7965 [==============================] - 13s - loss: 0.2688 - acc: 0.9033 - val_loss: 2.2827 - val_acc: 0.1797 - ETA: 2s - loss: 0.2881 - acc: 0.8959
Epoch 6/20
7965/7965 [==============================] - 13s - loss: 0.2498 - acc: 0.9257 - val_loss: 3.9045 - val_acc: 0.0602
Epoch 7/20
7965/7965 [==============================] - 13s - loss: 0.1802 - acc: 0.9471 - val_loss: 0.1990 - val_acc: 0.9116 - ETA: 5s - loss: 0.2063 - acc: 0.9404
Epoch 8/20
7965/7965 [==============================] - 13s - loss: 0.1558 - acc: 0.9505 - val_loss: 0.0958 - val_acc: 0.9674 - ETA: 3s - loss: 0.1682 - acc: 0.9489
Epoch 9/20
7965/7965 [==============================] - 13s - loss: 0.1494 - acc: 0.9552 - val_loss: 0.0441 - val_acc: 0.9890 - ETA: 11s - loss: 0.0543 - acc: 0.9824 - ETA: 11s - loss: 0.0915 - acc: 0.9734 - ETA: 4s - loss: 0.1651 - acc: 0.9513 - ETA: 2s - loss: 0.1508 - acc: 0.9558
Epoch 10/20
7965/7965 [==============================] - 13s - loss: 0.1266 - acc: 0.9599 - val_loss: 0.1376 - val_acc: 0.9413
Epoch 11/20
7965/7965 [==============================] - 13s - loss: 0.1108 - acc: 0.9612 - val_loss: 1.1236 - val_acc: 0.5537 - ETA: 10s - loss: 0.0994 - acc: 0.9640 - ETA: 8s - loss: 0.0998 - acc: 0.9610 - ETA: 6s - loss: 0.1060 - acc: 0.9597 - ETA: 2s - loss: 0.1110 - acc: 0.9609
Epoch 12/20
7965/7965 [==============================] - 13s - loss: 0.1372 - acc: 0.9630 - val_loss: 0.0362 - val_acc: 0.9895 - ETA: 9s - loss: 0.0893 - acc: 0.9727 - ETA: 5s - loss: 0.1270 - acc: 0.9604 - ETA: 0s - loss: 0.1376 - acc: 0.9628
Epoch 13/20
7965/7965 [==============================] - 13s - loss: 0.0965 - acc: 0.9687 - val_loss: 0.4876 - val_acc: 0.8549 - ETA: 11s - loss: 0.1321 - acc: 0.9517 - ETA: 6s - loss: 0.1091 - acc: 0.9622 - ETA: 4s - loss: 0.1019 - acc: 0.9650 - ETA: 3s - loss: 0.1046 - acc: 0.9651
Epoch 14/20
7965/7965 [==============================] - 13s - loss: 0.1122 - acc: 0.9701 - val_loss: 0.0282 - val_acc: 0.9905 - ETA: 9s - loss: 0.1020 - acc: 0.9654  - ETA: 5s - loss: 0.1063 - acc: 0.9692 - ETA: 4s - loss: 0.1260 - acc: 0.9666
Epoch 15/20
7965/7965 [==============================] - 13s - loss: 0.1086 - acc: 0.9735 - val_loss: 0.0233 - val_acc: 0.9930 - ETA: 11s - loss: 0.0457 - acc: 0.9896 - ETA: 10s - loss: 0.1181 - acc: 0.9732
Epoch 16/20
7965/7965 [==============================] - 13s - loss: 0.1061 - acc: 0.9736 - val_loss: 0.0242 - val_acc: 0.9910 - ETA: 9s - loss: 0.1054 - acc: 0.9603  - ETA: 6s - loss: 0.1340 - acc:0.9675 - ETA: 0s - loss: 0.1073 - acc: 0.9733
Epoch 17/20
7965/7965 [==============================] - 13s - loss: 0.0965 - acc: 0.9684 - val_loss: 0.1921 - val_acc: 0.9322 - ETA: 9s - loss: 0.0777 - acc: 0.9786 - ETA: 6s - loss: 0.0934 - acc: 0.9709 - ETA: 2s - loss: 0.1049 - acc: 0.9665
Epoch 18/20
7965/7965 [==============================] - 13s - loss: 0.0991 - acc: 0.9751 - val_loss: 0.0482 - val_acc: 0.9839 - ETA: 7s - loss: 0.0952 - acc: 0.9791 - ETA: 5s - loss: 0.0881 - acc: 0.9778 - ETA: 2s - loss: 0.0955 - acc: 0.9770 - ETA: 1s - loss: 0.0910 - acc: 0.9774
Epoch 19/20
7965/7965 [==============================] - 13s - loss: 0.1100 - acc: 0.9723 - val_loss: 0.0385 - val_acc: 0.9859 - ETA: 8s - loss: 0.0965 - acc: 0.9778 - ETA: 4s - loss: 0.1208 - acc: 0.9725 - ETA: 4s - loss: 0.1144 - acc: 0.9735 - ETA: 0s - loss: 0.1149 - acc: 0.9710
Epoch 20/20
7965/7965 [==============================] - 13s - loss: 0.1146 - acc: 0.9723 - val_loss: 0.0797 - val_acc: 0.9674 - ETA: 4s - loss: 0.1178 - acc: 0.9706```

```python
def plot_learning_curve(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./accuracy_curve.png')
    plt.clf()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./loss_curve.png')

plot_learning_curve(history)
```
![Retry?](...%0A)

```python
from sklearn.metrics import accuracy_score

print('Predicting on test data')
y_pred = np.rint(model.predict(X_test))

print(accuracy_score(y_test, y_pred))
```
```bash
Predicting on test data
0.985915492958
```
```python
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))
```
```bash
[[61  0]
 [ 1  9]]
```
```python
false_positive_mononuclear = np.intersect1d(np.where(y_pred == 1), np.where(y_test == 0))
img = X_test[false_positive_mononuclear[0]]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
```
```bash
<matplotlib.image.AxesImage at 0x7f462954dc50>
```
![Retry?](https://lh3.googleusercontent.com/KhTzHtNcgBbkzLTSX3Z1BSVszxhnefC3zt96dAFKFzrk9dTqBgANYpgWP1ETfiCtiDFhtA=s200)

```python
true_positive_mononuclear = np.intersect1d(np.where(y_pred == 1), np.where(y_test == 1))
img = X_test[true_positive_mononuclear[0]]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
img = X_test[true_positive_mononuclear[5]]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
img = X_test[true_positive_mononuclear[8]]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
true_positive_polynuclear = np.intersect1d(np.where(y_pred == 0), np.where(y_test == 0))
img = X_test[true_positive_polynuclear[21]]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
img = X_test[true_positive_polynuclear[53]]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
```

```python
img = X_test[true_positive_polynuclear[16]]
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
```

```bash
<matplotlib.image.AxesImage at 0x7f462954dc50>
```
![Retry?](https://lh3.googleusercontent.com/iOyKAUb7cSX2t8h5DTVfQGr8irSfQK5G9PI940KIqGl7hhTsQI6iqFuBzTZmlfS4z9Gn=s200)

# © Copyright 2018 Aaron Ma. All Rights Reserved. #